In [1]:
import numpy as np
import matplotlib.pyplot as plt
from helper import *
import scipy.sparse as sp
from test import load_data_sparse
from SGD import * 
from ALS import *
from Visualization import cv_result
from sklearn.linear_model import LinearRegression
from crossval import *

from surprise_models import *
from surprise.model_selection import train_test_split

In [2]:
data_name = "data/47b05e70-6076-44e8-96da-2530dc2187de_data_train.csv"

sp_ratings, pd_ratings = load_data_sparse(data_name, False)
spr_data = pandas_to_surprise(pd_ratings)

print(pd_ratings.shape) # (1176952, 3)

(1176952, 3)


In [3]:
trainset, testset = train_test_split(spr_data, test_size=.2)
#trainset = spr_data.build_full_trainset()

In [4]:
test_name = "data/sampleSubmission.csv"
_, pd_pred = load_data_sparse(test_name, False)
pred = pandas_to_surprise(pd_pred, True)

## New surprise models

In [5]:
svd_test, svd_final, rmse = surprise_SVD(trainset, testset, pred)

RMSE: 1.0047


In [6]:
so_test, so_final, so_rmse = surprise_slopeOne(trainset, testset, pred)

RMSE: 0.9988


In [7]:
num_features = 40  # K in the lecture notes
lambda_user = 0.1
lambda_film = 0.1
stop_criterion = 1e-3

In [8]:
als_test, als_final = ALS_CV(trainset, testset, pred, num_features, lambda_user, lambda_film, stop_criterion)

learn the matrix factorization using ALS...


In [9]:
lambda_user = 0.01
lambda_film = 0.01

In [10]:
sgd_test, sgd_final = matrix_factorization_SGD_CV(trainset, testset, pred, num_features, lambda_user, lambda_film, stop_criterion)

learn the matrix factorization using SGD...


In [11]:
print(svd_test.shape)
print(so_test.shape)

(235391,)
(235391,)


Put all the models in one matrix for the blending. This process will allow us to determmine the weight for each model.

In [12]:
X = [svd_test,so_test,als_test,sgd_test]
X = pd.DataFrame(X)
X.shape

(4, 235391)

In [13]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,235381,235382,235383,235384,235385,235386,235387,235388,235389,235390
0,3.846320,3.572313,4.084354,4.402332,3.742119,3.895190,2.778090,3.518556,4.656772,4.189647,...,4.440764,3.531467,3.375847,4.464314,3.418055,3.589066,3.647223,4.048169,4.088891,3.856305
1,4.080212,4.026579,4.110219,4.355211,4.007597,3.721890,2.990691,3.238610,4.746690,4.414452,...,4.568825,3.885620,3.586539,4.203199,3.690015,3.499964,3.708861,3.913917,4.205402,3.663710
2,3.665994,3.729448,4.010474,4.162216,3.883843,3.817786,3.470983,3.306738,4.532386,4.349971,...,4.431613,3.454259,3.474649,4.188209,3.314951,3.449810,3.606368,3.772140,4.247616,3.739809
3,4.159983,4.137781,4.313875,4.890797,3.927839,3.771009,3.049953,2.952903,4.675364,4.368986,...,4.504977,3.867032,3.502169,4.034563,3.648357,3.411801,3.248809,3.487351,4.195520,3.523803


## Blending

In [14]:
_,_,y = get_testset_indices(testset)

In [15]:
type(y)

tuple

In [16]:
#X = np.transpose(prediction)
X = np.transpose(X)
X[0:5]

,0,1,2,3
0,3.846320,4.080212,3.665994,4.159983
1,3.572313,4.026579,3.729448,4.137781
2,4.084354,4.110219,4.010474,4.313875
3,4.402332,4.355211,4.162216,4.890797
4,3.742119,4.007597,3.883843,3.927839


#### Linear Regression

In [17]:
#clf = LinearRegression()
#clf = clf.fit(X, y)
#print('Weights of the different models:', clf.coef_)
#clf.score(X, y)

#### Ridge Regression

In [18]:
from sklearn.linear_model import Ridge
linreg = Ridge(alpha=0.1, fit_intercept=False)
linreg.fit(X, y)
linreg.coef_

array([ 0.05640034, -0.10239242,  1.05599209,  0.02808248])

In [19]:
linreg.score(X, y)

0.22847488617760003

In [20]:
# compute MSE - Blending
sum((np.round(X.dot(linreg.coef_)) - y)**2)/y.shape[0]

AttributeError: 'tuple' object has no attribute 'shape'

In [ ]:
# compute MSE - SGD
sum(np.round(prediction_m[0] - y)**2)/y.shape[0]

In [ ]:
# compute MSE - ALS
sum(np.round(prediction_m[1] - y)**2)/y.shape[0]

## Manual comparison of ratings

In [21]:
print(X[0:10])

          0         1         2         3
0  3.846320  4.080212  3.665994  4.159983
1  3.572313  4.026579  3.729448  4.137781
2  4.084354  4.110219  4.010474  4.313875
3  4.402332  4.355211  4.162216  4.890797
4  3.742119  4.007597  3.883843  3.927839
5  3.895190  3.721890  3.817786  3.771009
6  2.778090  2.990691  3.470983  3.049953
7  3.518556  3.238610  3.306738  2.952903
8  4.656772  4.746690  4.532386  4.675364
9  4.189647  4.414452  4.349971  4.368986


In [22]:
print(y[0:10])

(5.0, 5.0, 2.0, 5.0, 5.0, 3.0, 3.0, 4.0, 5.0, 5.0)


In [23]:
# final result
result = X.dot(linreg.coef_)
print(result[0:10])

0    3.787234
1    3.843655
2    4.165677
3    4.334965
4    4.012321
5    3.976047
6    3.601442
7    3.441653
8    4.694078
9    4.500518
dtype: float64


## Determine the final prediction 
<p> This process is done using the previously found weights

In [24]:
#[svd_test,so_test,als_test,sgd_test]
result = svd_final * linreg.coef_[0] + so_final * linreg.coef_[1] + als_final * linreg.coef_[2] + sgd_final * linreg.coef_[3]

In [25]:
result.shape

(1176952,)

## Submission file

In [26]:
def create_submission_from_prediction(prediction, output_name):

    def prediction_transformed(prediction, ids_process):
        """ return the prediction transformed for the submission """
        y = []
        for i in range(len(ids_process)):
            row = ids_process[i][0]
            col = ids_process[i][1]
            y.append(prediction[row - 1, col - 1])
        return y

    def create_csv_submission(ids, y_pred, name):
        """
        Creates an output file in csv format for submission to kaggle
        Arguments: ids (event ids associated with each prediction)
                   y_pred (predicted class labels)
                   name (string name of .csv output file to be created)
        """
        with open(name, 'w') as csvfile:
            fieldnames = ['Id', 'Prediction']
            writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
            writer.writeheader()
            for r1, r2 in zip(ids, y_pred):
                writer.writerow({'Id': str(r1), 'Prediction': float(r2)})

    def round(x):
        if (x < 1):
            return 1
        elif (x > 5):
            return 5
        else:
            return x

    def transform(ids_txt):
        """ split the text index"""

        def deal_line(line):
            pos, rating = line.split(',')
            return str(pos)

        ids = [deal_line(line) for line in ids_txt]
        return ids

    prediction = np.vectorize(round)(prediction)

    DATA_TEST_PATH = 'data/sampleSubmission.csv'

    ids_txt = read_txt(DATA_TEST_PATH)[1:]
    ids_process = [deal_line(line) for line in ids_txt]

    # prediction under the right format
    # y = prediction_transformed(prediction, ids_process)
    y = np.vectorize(round)(prediction)
    
    y = np.rint(y)

    ids = transform(ids_txt)
    OUTPUT_PATH = output_name
    create_csv_submission(ids, y, OUTPUT_PATH)

In [27]:
create_submission_from_prediction(result, "out-4modelsurprise.csv")

#### 17 décembre 19:51

With : trainset, testset = train_test_split(spr_data, test_size=.2)
<p> And : X = [svd_test,so_test,als_test,sgd_test]
<p> ==> Weights (Ridge Regression, alpha=0.1) : array([ 0.05640034, -0.10239242,  1.05599209,  0.02808248])
<P>    
<p> RMSE : 1.025